<a href="https://colab.research.google.com/github/dkurbatovv/Python/blob/main/House_sales_in_king_country.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score, confusion_matrix, accuracy_score, log_loss, classification_report, mean_squared_error, mean_absolute_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor


from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor


In [ ]:
df = pd.read_csv('kc_house_data.csv')

In [ ]:
df.head()

In [ ]:
df = df.drop('id', axis = 1)

In [ ]:
df.head()

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.head()

In [ ]:
df['renovated'] = df['yr_renovated'].apply(lambda yr: 0 if yr == 0 else 1)

df = df.drop('date', axis=1)
df = df.drop('yr_renovated', axis=1)
df = df.drop('yr_built', axis=1)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df = df.drop(['lat', 'long'], axis = 1)

In [ ]:
df = df.drop('zipcode', axis = 1)

In [ ]:
plt.figure(figsize = (12,10))
sns.boxplot(data = df, x = 'bedrooms', y = 'price')

In [ ]:
plt.figure(figsize = (12,10))
sns.boxplot(data = df, x = 'floors', y = 'price')

In [ ]:
df.head()

In [ ]:
sns.countplot(data = df, x = 'renovated')

In [ ]:

features = ["sqft_lot", "sqft_above", "sqft_basement", "sqft_living15", "sqft_lot15"]

for i in features:
  sns.jointplot(x = i, y="price", data = df, kind = 'reg', size = 5)
  plt.show()

In [ ]:
plt.figure(figsize = (20,20))
sns.heatmap(df.corr(), cmap = 'icefire', annot = True)

In [ ]:
X = df.drop(['price', 'renovated'], axis = 1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 50)

print(X_train.shape, y_train.shape)

In [ ]:
X.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

In [ ]:
print('The training accuracy is ', lr.score(X_train, y_train))
print('The testing accuracy is ', lr.score(X_test, y_test))

In [ ]:
clf = RandomForestRegressor()
rfe = RFE(estimator = clf, n_features_to_select = 10, step = 1)
rfe.fit(X, y)
print('The best 10 features is :', X.columns[rfe.support_])

In [ ]:
best_features = list(X.columns[rfe.support_])

In [ ]:
X_rfe = df[best_features]

X_train, X_test, y_train, y_test = train_test_split(X_rfe, y, test_size = 0.3, random_state = 10)

In [ ]:
X_train

In [ ]:
lr_new = LinearRegression()
lr_new.fit(X_train, y_train)

y_pred_train = lr_new.predict(X_train)


In [ ]:
print('The Accuracy  on the training dataset is: ', lr_new.score(X_train, y_train) )
print('The Accuracy n2  on the training dataset is: ',r2_score(y_train, y_pred_train) )

# ***Decision Tree***

In [ ]:
dt_regressor = DecisionTreeRegressor(random_state=0)
dt_regressor.fit(X_train,y_train)

y_train_pred = dt_regressor.predict(X_train)
y_test_pred = dt_regressor.predict(X_test)

dt_regressor.score(X_test, y_test)

# ***Random Forest Regressor***

In [ ]:
Rf = RandomForestRegressor(n_estimators = 15,
                              criterion = 'mse',
                              random_state = 20,
                              n_jobs = -1)
Rf.fit(X_train,y_train)
Rf_train_pred = Rf.predict(X_train)
Rf_test_pred = Rf.predict(X_test)


print(r2_score(y_test,Rf_test_pred), Rf.score(X_test, y_test))

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
#from catboost import CatBoostRegressor

# ***GradientBoostingRegressor***

In [ ]:
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=100, learning_rate =.2)
gbr.fit(X_train, y_train)

y_pred_gbr = gbr.predict(X_train)

In [ ]:
gbr.score(X_test, y_test)

# ***XGBRegressor***

In [ ]:
xgb = XGBRegressor(max_depth = 3, n_estimators=50, learning_rate =.2)
xgb.fit(X_train, y_train)
y_train = xgb.predict(X_train)

print(xgb.score(X_test, y_test))

#***Вывод:***

Для получения более точных результатов можно попробовать убрать выбросы, или попробовать другое усреднение данных, к примеру StandardScaler.

Другой вариант решения, сделать нейросеть, и попробвать выполнить предсказания спомощью нее.

Так же следует попробовать Catboost.

